# 영화 추천: ranking 단계

실제 추천 시스템은 종종 두 단계로 구성됩니다.

1. 검색 단계(Retrieval Phase)에서는 가능한 모든 후보 중에서 수백개의 초기 후보 집합을 선택합니다. 이 모델의 주요 목적은 사용자가 관심이 없는 모든 후보를 효율적으로 제거하는 것입니다. 검색 모델은 수백만 개의 후보를 처리할 수 있으므로 계산적으로 효율적이어야 합니다.  

2. 순위 단계(Ranking Phase)에서는 검색 모델의 출력을 가져와서 가능한 한 가장 좋은 추천 항목을 선택하도록 미세 조정합니다. 이 작업은 사용자가 관심을 가질 만한 항목 집합을 가능한 후보의 최종 목록으로 좁히는 것입니다.

이 notebook 은 두번째 단계인 순위 단계 모델을 구현합니다.  

다음을 수행합니다.

1. 데이터를 가져와 훈련 세트와 테스트 세트로 나눕니다.
2. 순위 모델을 구현합니다.
3. 피팅하고 평가합니다.


In [ ]:
# TFRS와 호환되는 이전 version 설치
# 꼭 다시 시작한 직후 실행하세요!

In [ ]:
# 데이터를 보기 좋게 출력하기 위한 모듈
# 임시 파일 및 디렉토리 생성을 위한 모듈
# 타입 힌트를 위한 모듈
# TensorFlow 데이터셋을 사용하기 위한 라이브러리
# 추천 시스템을 구축하기 위한 TensorFlow 확장 라이브러리
# Version 확인

## Preparing the dataset

[retrieval](basic_retrieval) 과 동일한 데이터를 사용합니다. 이번에는 등급(explicit feedback)도 유지합니다. 이는 우리가 예측하려는 target 입니다.

In [ ]:
# MovieLens 100K 데이터셋에서 'ratings' 정보를 로드합니다. 데이터는 훈련용 분할로 지정됩니다.
# 로드된 'ratings' 데이터에서 필요한 특성만 추출하여 새로운 딕셔너리 구조로 매핑합니다.
# 'map' 함수를 사용하여 각 데이터 요소를 원하는 형태로 변환합니다.

이전과 마찬가지로 train/test를 80% 대 20%로 분할합니다.

In [ ]:
# 랜덤 시드를 설정
# ratings 데이터셋을 100,000개의 버퍼 크기로 무작위로 섞습니다.
# reshuffle_each_iteration=False는 매 반복(iteration)마다 데이터셋을 다시 섞지 않도록 설정합니다.
# 섞인 데이터셋에서 첫 80,000개를 훈련 데이터로 가져옵니다.
# 섞인 데이터셋에서 첫 80,000개 이후의 데이터 중 20,000개를 테스트 데이터로 가져옵니다.
# 훈련 데이터셋과 테스트 데이터셋의 길이를 계산합니다.
# TensorFlow 데이터셋에는 len() 함수를 직접 사용할 수 없으므로, cardinality() 메서드를 사용하여 각 데이터셋의 크기를 확인합니다.

In [ ]:
# ratings 데이터셋에서 영화 제목을 추출합니다. 대량의 데이터를 처리하기 위해 큰 배치 크기를 설정합니다.
# ratings 데이터셋에서 사용자 ID를 추출합니다. 마찬가지로 큰 배치 크기를 설정하여 처리합니다.
# 영화 제목과 사용자 ID의 중복을 제거합니다.
# TensorFlow 데이터셋을 list로 변환한 후 NumPy의 unique 함수를 사용하여 중복을 제거합니다.
# 중복 제거 후의 영화 제목 중 처음 10개를 출력합니다.

## 모델 구현

### Architecture

랭킹 모델은 검색 모델처럼 효율성 제약에 직면하지 않으므로 아키텍처를 선택할 때 조금 더 자유로워집니다.

여러 스택의 dense layer로 구성된 모델은 task 순위 지정을 위한 비교적 일반적인 아키텍처입니다. 다음과 같이 구현할 수 있습니다.

In [ ]:
class RankingModel(tf.keras.Model):
  def __init__(self):
    # 임베딩 차원을 32로 설정
    # 사용자에 대한 임베딩
        # 사용자 ID를 정수 인덱스로 변환
        # 사용자 임베딩 레이어
    # 영화에 대한 임베딩
      #  영화 제목을 정수 인덱스로 변환
      # 영화 임베딩 레이어
    # 평점 예측 신경망
  def call(self, inputs):
    # 입력받은 사용자 ID와 영화 제목에 대한 임베딩을 가져옵니다.
    # 사용자 임베딩과 영화 임베딩을 연결(concatenate)하고 평점을 예측

이 모델은 user ID와 movie title을 가져와 예상 등급을 출력합니다.

In [ ]:
# RankingModel 인스턴스를 생성하고, 사용자 ID와 영화 제목을 입력으로 전달하여 호출

### Loss and metrics

다음 구성 요소는 모델을 훈련하는 데 사용되는 손실입니다. TFRS에는 이를 쉽게 하기 위한 여러 손실 계층과 작업이 있습니다.

이 경우 손실 함수와 메트릭 계산을 함께 묶는 편리한 래퍼인 `Ranking` 작업 개체를 사용합니다.

평점을 예측하기 위해 `MeanSquaredError` Keras 손실과 함께 사용할 것입니다.

In [ ]:
# 순위 매기기 작업을 정의
  # 손실 함수로 평균 제곱 오차(Mean Squared Error)를 사용합니다.
  # 이는 예측 평점과 실제 평점 간의 차이의 제곱에 대한 평균을 계산합니다.
  # 평가 지표로는 루트 평균 제곱 오차(Root Mean Squared Error)를 사용합니다.
  # RMSE는 MSE의 제곱근으로, 오차의 크기를 원래의 단위로 복원하여 해석하기 쉽게 만듭니다.

task 자체는 true 와 predicted 를 인수로 받아 계산된 손실을 반환하는 Keras 계층입니다. 이를 사용하여 모델의 훈련 루프를 구현합니다.

### The full model

이제 모든 것을 하나의 모델로 통합할 수 있습니다. TFRS는 빌딩 모델을 간소화하는 base 모델 클래스(`tfrs.models.Model`)를 노출합니다. feature를 제공하고 손실 값을 반환합니다.

그러면 base model이 우리 모델에 맞는 적절한 훈련 루프를 생성합니다.

In [ ]:
# 사용자가 영화에 부여할 평점을 예측하는 Full Model
class MovielensModel(tfrs.models.Model):
  def __init__(self):
    # RankingModel 클래스를 사용하여 순위 매기기 모델을 정의
    # 순위 매기기 작업을 정의합니다. 손실 함수와 평가 지표를 지정합니다.
       # 손실 함수로 평균 제곱 오차를 사용
       # 평가 지표로 RMSE를 사용
  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    # 모델에 입력으로 사용자 ID와 영화 제목을 전달하여 평점 예측을 수행
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # 실제 사용자 평점을 가져옵니다.
    # 모델을 호출하여 평점 예측을 수행
    # 순위 매기기 작업(task)을 사용하여 손실을 계산
    # 이는 예측 평점과 실제 평점 간의 손실 및 평가 지표를 계산

## Fitting and evaluating

훈련 및 평가 데이터를 섞고 일괄 처리하고 캐시합니다.

In [ ]:
# 훈련 데이터셋을 섞고 배치 처리한 후 캐시합니다.
# 배치 처리는 모델 학습이나 평가 시 여러 샘플을 동시에 처리하도록 하며, 캐싱은 반복 평가 시 성능을 향상시킵니다.

 모델을 훈련합니다.

RMSE 측정항목이 낮을수록 모델이 등급을 예측하는 경우가 더 정확해집니다.

## Testing the ranking model

이제 영화 세트에 대한 예측을 계산하여 ranking model을 테스트한 다음 예측을 기반으로 이러한 영화의 순위를 지정할 수 있습니다.


In [ ]:
# 추천 시스템에서 검색 모델을 통해 얻은 추천 영화 후보 목록입니다.
# 각 영화 제목에 대해 평가 모델을 사용하여 사용자 42의 평점을 예측합니다.
    # 평가 모델에 사용자 ID와 영화 제목을 입력하여 평점을 예측합니다.
# 예측된 평점을 출력합니다. 출력은 평점이 높은 순으로 정렬됩니다.